<h1 style="color: #9370DB;"> Stock Tracker Cleaning API </h1>

### Yahoo Finance API:

**First impressions:**

Our **project goal** is to identify the performance of various sectors in the S&P 500. After reading the [documentation](https://rapidapi.com/nusantaracodedotcom/api/yahoo-finance-api-data) we will proceed with the following **strategy**:

<h2 style="color: #4C9F70;">01 | Data Extraction </h2>

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

api_key = os.getenv("x-rapidapi-key")

In [3]:
url = "https://yahoo-finance-api-data.p.rapidapi.com/summary/symbol-info"

querystring = {"symbol":"AAPL"}

headers = {
	"x-rapidapi-key": api_key,
	"x-rapidapi-host": "yahoo-finance-api-data.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'success': True, 'code': 200, 'message': 'sucess', 'data': [{'meta': {'symbol': ['AAPL'], 'type': ['sigdev_performance', ' sigdev_corporate_guidance', ' sigdev_ownership_control', ' sigdev_financing', ' sigdev_corporate_deals']}, 'timestamp': [1430107200, 1430107200, 1437451200, 1437451200, 1445918400, 1453784400, 1464840000, 1464926400, 1467259200, 1469505600, 1469505600, 1469592000, 1469678400, 1469678400, 1473048000, 1477368000, 1477368000, 1477368000, 1477454400, 1482210000, 1482296400, 1482382800, 1482382800, 1485838800, 1485838800, 1485925200, 1488430800, 1491796800, 1491883200, 1492574400, 1492574400, 1493697600, 1494475200, 1494993600, 1500436800, 1500436800, 1500436800, 1500609600, 1501560000, 1505102400, 1505361600, 1506052800, 1507262400, 1509508800, 1509595200, 1509595200, 1509595200, 1509681600, 1512363600, 1513054800, 1513227600, 1515042000, 1517374800, 1517374800, 1517461200, 1517461200, 1519448400, 1521086400, 1522209600, 1524542400, 1525147200, 1525147200, 1532491200,

In [ ]:
# URL of the Wikipedia page containing the list of S&P 500 companies
web = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

tables = pd.read_html(web)
df = tables[0] # The first table on the page contains the list of S&P 500 companies
lista = df['Symbol'].tolist()
lista

In [ ]:
len(lista)

In [ ]:
# Info empresas. 
url = "https://yahoo-finance-api-data.p.rapidapi.com/summary/symbol-profile"
headers = {
    "x-rapidapi-key": api_key,
    "x-rapidapi-host": "yahoo-finance-api-data.p.rapidapi.com"
}

# Data container for the results
all_data = []

# Loop through each symbol in the list
for symbol in lista[:503]:
    querystring = {"symbol": symbol}
    
    # Send the request for the current symbol
    response = requests.get(url, headers=headers, params=querystring)
    print(response)
    # Check if the request was successful
    if response.status_code == 200:
        status_data = response.json()

        # Ensure the 'data' key exists
        if 'data' in status_data:
            company_data = status_data['data']
            
            # Extracting required fields into a dictionary
            company_info = {
                'symbol': symbol,  # Include the stock symbol for identification
                'address1': company_data.get('address1'),
                'city': company_data.get('city'),
                'state': company_data.get('state'),
                'zip': company_data.get('zip'),
                'country': company_data.get('country'),
                'phone': company_data.get('phone'),
                'website': company_data.get('website'), 
                'industry': company_data.get('industry'),
                'industryKey': company_data.get('industryKey'), 
                'industryDisp': company_data.get('industryDisp'),
                'sector': company_data.get('sector'),
                'sectorKey': company_data.get('sectorKey'),
                'sectorDisp': company_data.get('sectorDisp'),
                'longBusinessSummary': company_data.get('longBusinessSummary'),
                'fullTimeEmployees': company_data.get('fullTimeEmployees'),
                'maxAge': company_data.get('maxAge')
            }

            # Add company_info to the list
            all_data.append(company_info)
        else:
            print(f'Error: No data found for symbol {symbol}.')
    else:
        print(f'Error: Failed to retrieve data for symbol {symbol}.')

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(all_data)

In [ ]:
df

In [ ]:
df.to_csv("info_250_500.csv")

In [ ]:
# df.to_csv("info_0_250.csv")

In [ ]:
# Historico Precios 
url = "https://yahoo-finance-api-data.p.rapidapi.com/chart/simple-chart"

headers = {
	"x-rapidapi-key": api_key,
	"x-rapidapi-host": "yahoo-finance-api-data.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
data = response.json()
all_data2 = {
    "symbol": [],
    "timestamps":[],
    "price":[],
}
all_data2_df = pd.DataFrame(all_data2)
all_data3 = []

# Loop through each symbol in the list
for symbol in lista[:503]:
    querystring = {"symbol":symbol,"limit":"10","range":"max"}
    
    # Send the request for the current symbol
    response = requests.get(url, headers=headers, params=querystring)
    print(response)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Ensure the 'data' key exists
        if ('data' in data) & ("timestamp" in data["data"][0]) & ("adjclose" in data["data"][0]['indicators']["adjclose"][0]) & ("meta" in data["data"][0]):
            profile = data["data"][0]['meta']
            dates = data["data"][0]["timestamp"]  # Extract timestamps
            close_prices = data["data"][0]['indicators']["adjclose"][0]["adjclose"]  # Extract closing prices
        
            # Para sacer info de simbolo, tiempo y precio. 
            historical = {
                "symbol": profile['symbol'], 
                "timestamps": dates, 
                "price": close_prices,
            }
            # Para sacar info de perfil de accion. 
            company_info2 = {
                "currency":profile['currency'],
                'symbol': symbol,
                "exchangeName":profile['exchangeName'],
                "fullExchangeName":profile["fullExchangeName"],
                "instrumentType":profile['instrumentType'],
                "firstTradeDate":profile['firstTradeDate'],
                "regularMarketTime":profile['regularMarketTime'],
                "hasPrePostMarketData":profile['hasPrePostMarketData'],
                "gmtoffset":profile['gmtoffset'],
                "timezone":profile['timezone'],
                "exchangeTimezoneName":profile['exchangeTimezoneName'],
                "regularMarketPrice":profile['regularMarketPrice'],
                "fiftyTwoWeekHigh":profile['fiftyTwoWeekHigh'],
                "fiftyTwoWeekLow":profile['fiftyTwoWeekLow'],
                "regularMarketDayHigh":profile['regularMarketDayHigh'],
                "regularMarketDayLow":profile['regularMarketDayLow'],
                "regularMarketVolume":profile['regularMarketVolume'],
                "longName":profile['longName'],
                "shortName":profile['shortName'],
                "chartPreviousClose":profile['chartPreviousClose'],
                "priceHint":profile['priceHint'],
            }

            #Conver dicirtonary to Dataframe. 
            historical_df = pd.DataFrame(historical)
            # Storage 
            all_data2_df = pd.concat([all_data2_df, historical_df])
            all_data3.append(company_info2)

            print(f'Data successfully fetched for {symbol}')
        else: 
            print(f'No valid data found for {symbol}')

    else: 
        print(f'Error for {symbol} {response}')




In [ ]:
all_data2_df

In [ ]:
all_data2_df.to_csv("historical_prices.csv")

In [ ]:
all_data2_df

In [ ]:
all_data3_df = pd.DataFrame(all_data3)
all_data3_df

In [ ]:
all_data3_df.to_csv('extrainfo.csv')

In [ ]:
# Double check what can be cleaned. 
# Assuming the API returns time-series data:
data = response.json()
profile = data["data"][0]['meta']
dates = data["data"][0]["timestamp"]  # Extract timestamps
close_prices = data["data"][0]['indicators']["adjclose"][0]["adjclose"]  # Extract closing prices

In [ ]:
# Double check but can be cleaned. 
historical = {
    "symbol": profile['symbol'],
    "timestamps" : dates, 
    "price" : close_prices,
}
historical_df = pd.DataFrame(historical)
historical_df

In [ ]:
# Important to clean the timestaps. 
all_data2_df['timestamps'] = pd.to_datetime(all_data2_df["timestamps"], unit="s")
all_data2_df

In [ ]:
all_data2_df.to_csv("historical_prices_final.csv")

In [ ]:
url = "https://yahoo-finance-api-data.p.rapidapi.com/chart/simple-chart"

querystring = {"symbol": "AAPL", "limit": "10", "range": "max"}

headers = {
    "x-rapidapi-key": api_key,
    "x-rapidapi-host": "yahoo-finance-api-data.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# Convert response to JSON
data = response.json()

# Assuming the API returns time-series data:
if "chart" in data and "result" in data["chart"]:
    results = data["chart"]["result"][0]  # Navigate inside the JSON structure

    timestamps = results["timestamp"]  # Extract timestamps
    close_prices = results["indicators"]["quote"][0]["close"]  # Extract closing prices

    # Create a DataFrame
    df = pd.DataFrame({
        "Date": pd.to_datetime(timestamps, unit="s"),  # Convert timestamps to readable dates
        "Close Price": close_prices
    })

    print(df)
    df.to_csv("apple_stock_data.csv", index=False)  # Save to CSV file

else:
    print("Unexpected data format!")
